In [5]:
import pandas as pd 
import numpy as np 
import random

In [6]:
data = pd.read_csv(r'car_fuel_efficiency.csv')

In [7]:
data.isna().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [8]:
data2 = data[['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']]

In [9]:
data2

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,170,159.0,3413.433759,2003,13.231729
1,130,97.0,3149.664934,2007,13.688217
2,170,78.0,3079.038997,2018,14.246341
3,220,NaN,2542.392402,2009,16.912736
4,210,140.0,3460.870990,2009,12.488369
...,...,...,...,...,...
9699,140,164.0,2981.107371,2013,15.101802
9700,180,154.0,2439.525729,2004,17.962326
9701,220,138.0,2583.471318,2008,17.186587
9702,230,177.0,2905.527390,2011,15.331551


In [10]:
datampg = data2['fuel_efficiency_mpg']

In [11]:
datampg.kurtosis()

np.float64(0.02273224449763367)

with  kurtosis of less than 3, It has a light tail and flatter peak

In [12]:
data2.isna().sum()

engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

In [13]:
data2['horsepower'].median()

np.float64(149.0)

In [14]:
np.random.seed(42)

In [15]:
n = len(data2)
n

n_val = int(n*0.2)
n_test =int(n*0.2)
n_train= n- n_test-n_val
print(f'n_val{n_val}\n ntes {n_test +n_val}\n {n_train}')

n_val1940
 ntes 3880
 5824


In [16]:
idx =np.arange(n)
np.random.shuffle(idx)

In [17]:
df_train = data2.iloc[idx[:n_train]]
df_val =data2.iloc[idx[n_train: n_train+n_val]]
df_test = data2.iloc[idx[n_train+n_val:]]


In [18]:
df_train=df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [19]:
w0 = 0 
w = [1,1,1]
xi = [23,56,35]

In [20]:
w_new = [w0] + w

In [21]:
def linear_regression(xi):
    xi= [1]+[xi]
    return np.dot(xi,w_new)


In [22]:
df_train.head(2)

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,220,144.0,2535.887591,2009,16.642943
1,160,141.0,2741.170484,2019,16.298377


In [23]:
df_train.shape[0]

5824

In [24]:
df_train.isna().sum()

engine_displacement      0
horsepower             429
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

# filling with  0

In [25]:
x_train = df_train.fillna(0)

In [26]:
y_train = df_train['fuel_efficiency_mpg'].values
y_train


array([16.64294342, 16.29837715, 18.59182197, ..., 17.59323665,
       18.92574819, 17.96528447], shape=(5824,))

In [27]:
t_x =x_train.drop('fuel_efficiency_mpg',axis=1)

In [28]:
t_x

,engine_displacement,horsepower,vehicle_weight,model_year
0,220,144.0,2535.887591,2009
1,160,141.0,2741.170484,2019
2,230,155.0,2471.880237,2017
3,150,206.0,3748.164469,2015
4,300,111.0,2135.716359,2006
...,...,...,...,...
5819,260,139.0,2606.972984,2009
5820,280,132.0,4004.214323,2014
5821,210,152.0,2500.175687,2020
5822,250,154.0,2254.313245,2002


In [29]:
t_x=t_x.values

In [30]:
t_x

array([[ 220.        ,  144.        , 2535.88759124, 2009.        ],
       [ 160.        ,  141.        , 2741.17048439, 2019.        ],
       [ 230.        ,  155.        , 2471.88023726, 2017.        ],
       ...,
       [ 210.        ,  152.        , 2500.17568746, 2020.        ],
       [ 250.        ,  154.        , 2254.31324473, 2002.        ],
       [ 210.        ,  152.        , 2435.20360628, 2004.        ]],
      shape=(5824, 4))

# Train the linear regression

In [31]:
def train_lin_regression(x,y):
    ones =np.ones(x.shape[0])
    x = np.column_stack([ones,x])
    xtx = x.T.dot(x)
    xtx_inv= np.linalg.inv(xtx)
    w_full = xtx_inv.dot(x.T).dot(y)
    return w_full[0],w_full[1:]

In [32]:
w0 , w = train_lin_regression(t_x,y_train)

In [33]:
w

array([ 9.93613218e-05,  3.54221891e-03, -5.01328015e-03,  3.42992869e-04])

In [34]:
y_pred = w0 + t_x.dot(w)
y_pred

array([17.33526223, 16.29296317, 17.69885099, ..., 17.54541307,
       18.78287539, 17.86564842], shape=(5824,))

In [35]:
x_val = df_val.drop('fuel_efficiency_mpg',axis=1)

x_test = df_test.drop('fuel_efficiency_mpg',axis=1)

In [36]:
t_val = x_val.fillna(0).values

In [37]:
t_val

array([[ 180.        ,  112.        , 2772.13441008, 2020.        ],
       [ 260.        ,  138.        , 3014.06152983, 2022.        ],
       [ 140.        ,  181.        , 3652.18615117, 2016.        ],
       ...,
       [ 230.        ,  202.        , 3303.00068812, 2008.        ],
       [ 220.        ,  169.        , 3293.42025107, 2008.        ],
       [ 200.        ,  117.        , 2692.7812918 , 2004.        ]],
      shape=(1940, 4))

In [38]:
w

array([ 9.93613218e-05,  3.54221891e-03, -5.01328015e-03,  3.42992869e-04])

In [39]:
y_pred = w0 + t_val.dot(w)

In [40]:
y_pred

array([16.0373382 , 14.92522236, 11.86445895, ..., 13.69560868,
       13.62575126, 16.44936805], shape=(1940,))

In [41]:
y_val = df_val['fuel_efficiency_mpg'].values
y_val

array([15.36634109, 14.60118181, 12.24703219, ..., 13.61312692,
       13.73334321, 15.81604972], shape=(1940,))

# Rmse

In [42]:
def rmse(y,ypredicted):
    se = (y-ypredicted)**2
    mse = se.mean()
    return np.sqrt(mse)

In [43]:
rmse0=round(rmse(y_val,y_pred),2)
rmse0

np.float64(0.52)

# Filling with mean 

In [44]:
def mean_drop(df):
    mx = df.drop('fuel_efficiency_mpg',axis=1)
    mxt = mx.fillna(df_train['horsepower'].mean()).values
    return mxt , mx

In [45]:
mean_xtrain,m_xtrain=mean_drop(df_train)
mean_xval,m_xval=mean_drop(df_val)

In [46]:
w0,w = train_lin_regression(mean_xtrain,y_train)

In [67]:
print(w0)
print(w)

0.01841801730804525
[ 0.00015161  0.00354204 -0.00500736  0.01465133]


In [47]:
y_train

array([16.64294342, 16.29837715, 18.59182197, ..., 17.59323665,
       18.92574819, 17.96528447], shape=(5824,))

In [48]:
y_pred = w0 + mean_xtrain.dot(w)
y_pred

array([17.25910991, 16.19011261, 17.6924588 , ..., 17.51666431,
       18.77812427, 17.84593445], shape=(5824,))

In [ ]:
w0=0.01841801730804525
w=[0.00015161,0.00354204,-0.00500736,0.01465133]
w =np.array(w)
y_pred = w0 + (np.array([listoffeatures])).dot(w)
y_pred

import streamlit as st
import numpy as np

# --------------------------
# MODEL WEIGHTS (Embedded)
# --------------------------
w0 = 0.01841801730804525
w = np.array([0.00015161, 0.00354204, -0.00500736, 0.01465133])

# --------------------------
# STREAMLIT UI
# --------------------------
st.title("🚗 Fuel Efficiency Prediction (MPG)")

st.write("Enter the vehicle details below to predict **Fuel Efficiency (Miles Per Gallon)**")

# Input fields (no min/max constraints)
engine_displacement = st.number_input("Engine Displacement (float)", format="%.4f")
horsepower = st.number_input("Horsepower (integer)", format="%d")
vehicle_weight = st.number_input("Vehicle Weight (float)", format="%.2f")
model_year = st.number_input("Model Year (integer)", format="%d")

# Prediction
if st.button("Predict"):
    features = np.array([engine_displacement, horsepower, vehicle_weight, model_year])
    y_pred = w0 + features.dot(w)
    st.success(f"Estimated Fuel Efficiency: **{y_pred:.2f} MPG**")


In [49]:
y_val

array([15.36634109, 14.60118181, 12.24703219, ..., 13.61312692,
       13.73334321, 15.81604972], shape=(1940,))

In [50]:
y_predval = w0 + mean_xval.dot(w)
y_predval

array([15.73837498, 14.80338305, 12.03624928, ..., 14.01482574,
       13.72161355, 16.1936294 ], shape=(1940,))

In [51]:
round(rmse(y_val,y_predval),2)

np.float64(0.46)

In [52]:
df_train

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,220,144.0,2535.887591,2009,16.642943
1,160,141.0,2741.170484,2019,16.298377
2,230,155.0,2471.880237,2017,18.591822
3,150,206.0,3748.164469,2015,11.818843
4,300,111.0,2135.716359,2006,19.402209
...,...,...,...,...,...
5819,260,139.0,2606.972984,2009,16.964054
5820,280,132.0,4004.214323,2014,10.059094
5821,210,152.0,2500.175687,2020,17.593237
5822,250,154.0,2254.313245,2002,18.925748


# Question 4

# Regularising

In [53]:
def train_lin_regression_reg(x,y,i):
    ones =np.ones(x.shape[0])
    x = np.column_stack([ones,x])
    xtx = x.T.dot(x)
    xtx = xtx + i * np.eye(xtx.shape[1])
    xtx_inv= np.linalg.inv(xtx)
    w_full = xtx_inv.dot(x.T).dot(y)
    return w_full[0],w_full[1:]


In [54]:
y_val

array([15.36634109, 14.60118181, 12.24703219, ..., 13.61312692,
       13.73334321, 15.81604972], shape=(1940,))

In [66]:
r=[0,0.01,0.1,1,5,10,100]
for i in r:
    count=0
    w0,w =train_lin_regression_reg(t_x,y_train,i)
    print(w0)
    print(w)
    
    y_pred = w0 + x_val.dot(w)
    rotmean=round(rmse(y_val,y_pred),4)
    print(f'rmse\t{i} is {rotmean}\n')
    count+=1

28.827365474598718
[ 9.93613218e-05  3.54221891e-03 -5.01328015e-03  3.42992869e-04]
rmse	0 is 3.5804

24.926838421084422
[ 0.00010643  0.0035422  -0.00501248  0.00228024]
rmse	0.01 is 3.5801

11.239661085047366
[ 0.00013125  0.00354212 -0.00500966  0.00907815]
rmse	0.1 is 3.5795

1.7315979412398264
[ 0.00014849  0.00354207 -0.00500771  0.01380046]
rmse	1 is 3.5795

0.36380750207153073
[ 0.00015098  0.00354206 -0.00500743  0.01447979]
rmse	5 is 3.5795

0.18306246622476907
[ 0.0001513   0.00354206 -0.00500739  0.01456956]
rmse	10 is 3.5795

0.01841801730804525
[ 0.00015161  0.00354204 -0.00500736  0.01465133]
rmse	100 is 3.5795



In [ ]:
28.827365474598718



# Question 5

In [56]:
# function that prepare x matric and y matrix
def mean_drop(df):
    mx = df.drop('fuel_efficiency_mpg',axis=1)
    mxt = mx.fillna(df_train['horsepower'].mean()).values
    return mxt , mx


In [57]:
data2

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,170,159.0,3413.433759,2003,13.231729
1,130,97.0,3149.664934,2007,13.688217
2,170,78.0,3079.038997,2018,14.246341
3,220,NaN,2542.392402,2009,16.912736
4,210,140.0,3460.870990,2009,12.488369
...,...,...,...,...,...
9699,140,164.0,2981.107371,2013,15.101802
9700,180,154.0,2439.525729,2004,17.962326
9701,220,138.0,2583.471318,2008,17.186587
9702,230,177.0,2905.527390,2011,15.331551


In [58]:
rscores = []
seedvalues= [0,1,2,3,4,5,6,7,8,9]
n = len(data2)
for seed in seedvalues:
    np.random.seed(seed)

    n_val = int(n*0.2)
    n_test =int(n*0.2)
    n_train= n- n_test-n_val

    idx =np.arange(n)
    np.random.shuffle(idx)
    data2 = data2.fillna(0)
    q5_train = data2.iloc[idx[:n_train]]
    q5_val =data2.iloc[idx[n_train: n_train+n_val]]
    q5_test = data2.iloc[idx[n_train+n_val:]]

    df_trains=q5_train.reset_index(drop=True).values
    df_vals = q5_val.reset_index(drop=True).values
    df_tests = q5_test.reset_index(drop=True).values


    y_trains = q5_train['fuel_efficiency_mpg'].values
    y_vals  = q5_val['fuel_efficiency_mpg'].values

    x_train =q5_train.drop('fuel_efficiency_mpg',axis=1).values
    x_val =q5_val.drop('fuel_efficiency_mpg',axis=1).values
    x_test =q5_test.drop('fuel_efficiency_mpg',axis=1).values
    w0,w =train_lin_regression(x_train,y_trains)
    y_pred = w0 + x_val.dot(w)
    rscore=rmse(y_val,y_pred)
    rscores.append(rscore)
    print(f'The rmse is {rscore}')

The rmse is 3.603312621323288
The rmse is 3.657915830364171
The rmse is 3.5957899828893285
The rmse is 3.7140819673960745
The rmse is 3.6010272237568883
The rmse is 3.6216982082365874
The rmse is 3.525709510228474
The rmse is 3.5962629512163433
The rmse is 3.5998758306075938
The rmse is 3.582493524185547


In [59]:
rscores=np.array(rscores)
rscoresstd = round(np.std(rscores),3)
rscoresstd

np.float64(0.047)

In [60]:


n = len(data2)

np.random.seed(9)

n_val = int(n*0.2)
n_test =int(n*0.2)
n_train= n- n_test-n_val

idx =np.arange(n)
np.random.shuffle(idx)
data2 = data2.fillna(0)
q5_train = data2.iloc[idx[:n_train]]
q5_val =data2.iloc[idx[n_train: n_train+n_val]]
q5_test = data2.iloc[idx[n_train+n_val:]]

df_trains=q5_train.reset_index(drop=True).values
df_vals = q5_val.reset_index(drop=True).values
df_tests = q5_test.reset_index(drop=True).values


y_trains = q5_train['fuel_efficiency_mpg'].values
y_vals  = q5_val['fuel_efficiency_mpg'].values
y_test  = q5_test['fuel_efficiency_mpg'].values

x_train =q5_train.drop('fuel_efficiency_mpg',axis=1)
x_val =q5_val.drop('fuel_efficiency_mpg',axis=1)
x_comb= np.vstack([x_train,x_val])
y_comb= np.hstack([y_trains,y_vals])

x_test =q5_test.drop('fuel_efficiency_mpg',axis=1).values
w0,w =train_lin_regression_reg(x_comb,y_comb,0.001)
y_pred = w0 + x_test.dot(w)
rscore=rmse(y_test,y_pred)
print(f'The rmse is {rscore}')

The rmse is 0.5156261299185628
